# Extractions

The objective of this notebook is to generate extractions from the verbatims.

In [4]:
import pandas as pd
import json
import importlib

from tqdm import tqdm
from typing import List, Dict
from pymongo import MongoClient

import boto3
import certifi

from utils.extractions_utils import generate_extraction_results, split_text_into_parts, extract_information_from_text, add_extractions_to_splitted_analysis

In [5]:
STAGE = 'prod'

Loading all key libraries

In [6]:
_secrets_manager_client = boto3.client("secretsmanager", region_name="eu-west-3")

_secrets = json.loads(
    _secrets_manager_client.get_secret_value(
        SecretId=f"{STAGE.capitalize()}/alloreview"
    )["SecretString"]
)
MONGO_CONNECTION_STRING = (
    "mongodb+srv://alloreview:{}@feedbacksdev.cuwx1.mongodb.net".format(
        _secrets["mongodb"]["password"]
    )
)

OPENAI_API_KEY = _secrets["openai"]["api_key"]
LLM_API_KEY = _secrets["litellm"]["api_key"]


In [7]:
mongo_client = MongoClient(MONGO_CONNECTION_STRING,tlsCAFile=certifi.where())

collection = mongo_client['feedbacks_db']['feedbacks_Prod']

Define the brand and a short description of the brand.

In [8]:
BRAND = 'ditp_analysis'

In [11]:
#NEW
from_mongo = pd.read_csv('ditp_test.csv')

In [10]:
from_mongo = pd.DataFrame(list(collection.aggregate([
    {
        '$match': {
            'brand': BRAND,
        },
    },
])))


from_mongo.shape

KeyboardInterrupt: 

## Run the extraction pipeline on sample

In [12]:
import importlib
# this function allows to parallelize the extraction process and to save the results on the mongo database
from utils.extractions_utils import process_extractions_in_parallel
module = importlib.import_module('utils.extractions_utils')
importlib.reload(module)

# Ensure the function is re-imported after reloading the module
process_extractions_in_parallel = module.process_extractions_in_parallel

In [13]:
subdf = from_mongo.sample(2)
print(f'Test will be done on {subdf.shape[0]} samples.')



Test will be done on 2 samples.


In [14]:
def format_ligne(ligne):
    # Fonction interne pour gérer les valeurs manquantes
    def extraire_champ(champ, allow_empty=False):
        return champ if pd.notnull(champ) and (allow_empty or champ != 'N/A') else None

    # Champs obligatoires et facultatifs
    champs = [
        ("Intitulé Structure 1", ligne.get("intitule_structure_1"), False),
        ("Intitulé Structure 2", ligne.get("intitule_structure_2"), True),
        ("Tags Métiers", ligne.get("tags_metiers"), True),
        ("Pays de la demande", ligne.get("pays"), False)
    ]
    
    # Initialisation des lignes avec une phrase fixe
    lignes = ["Feedbacks are from French public services."]
    
    # Génération des lignes dynamiques si les champs sont présents
    for label, champ, allow_empty in champs:
        valeur = extraire_champ(champ, allow_empty)
        if valeur:
            lignes.append(f"{label}: {valeur}")
    
    # Retour du résultat formaté
    return "\n".join(lignes)

In [15]:
#NEW
subdf['text'] = subdf['verbatims']
subdf['brand_context'] = subdf.apply(format_ligne, axis=1)
texts_with_ids = subdf[['text', '_id', 'brand_context']]

In [16]:
texts_with_ids

,text,_id,brand_context
79,Absence de réponse au problème\r\nSouhaitant c...,ditp_analysis/4548008,Feedbacks are from French public services.\nIn...
95,modification association\r\nSuite à une Assemb...,ditp_analysis/4567938,Feedbacks are from French public services.\nIn...


In [17]:
text = texts_with_ids.iloc[0]  # Assuming texts_with_ids is a DataFrame and you want the first row
extraction = extract_information_from_text(
    text.get('text'),
    text.get('_id'),
    text.get('brand_context'),
    language='french',
    model="gpt-4o-mini"
)
extraction

{'id': 'ditp_analysis/4548008',
 'splitted_analysis': [{'text': 'Absence de réponse au problème',
   'extractions': [{'sentiment': 'NEGATIVE',
     'extraction': 'Absence de réponse au problème'}]},
  {'text': 'Souhaitant céder mon véhicule',
   'extractions': [{'sentiment': 'POSITIVE',
     'extraction': 'Souhait de céder un véhicule'}]},
  {'text': 'je me connecte via France Connect au service ANTS',
   'extractions': [{'sentiment': 'NEGATIVE',
     'extraction': 'Problème de correspondance entre titulaire de compte et carte grise'}]},
  {'text': "je note le numéro d'immatriculation du véhicule"},
  {'text': 'et apparait un message surprenant comme quoi le titulaire du compte ne correspond pas au titulaire de la carte grise',
   'extractions': [{'sentiment': 'NEGATIVE',
     'extraction': 'Problème de correspondance entre titulaire de compte et carte grise'}]},
  {'text': "alors que c'est bien mon nom sur la carte grise",
   'extractions': [{'sentiment': 'NEGATIVE',
     'extraction'

In [3]:
texts_with_ids

NameError: name 'texts_with_ids' is not defined

In [18]:
extractions = process_extractions_in_parallel(
    texts_with_ids,
    brand_name=BRAND,
    language='french',
    model="gpt-4o-mini",
    save_to_mongo=False
)

Processing chunks: 100%|██████████| 1/1 [00:04<00:00,  4.18s/it]


## Checking result in MongoDB

In [12]:
# getting the document from the database to check if the extractions are saved
# matching the brand and the id in res

documents = collection.find({
    'brand': BRAND,
    '_id': {'$in': [r['id'] for r in extractions]}
})

documents = pd.DataFrame(documents)

In [13]:
documents.shape

(20, 84)

In [14]:
documents.sample().iloc[0].verbatims

"Téléphone\nPlus de 50 appels en une journée, jamais de réponse, personne ne décroche le téléphone, pourtant appels passés dès l'ouverture jusqu'à la fermeture sans interruption, toujours la même chose: les agents sont en communication, alors que l'appel est lancé même avant l'ouverture pour être sûr d'être partis les 1ers. Très mauvaise expérience, et quand il ou elle répond, ils ont vite envie de raccrocher sans pour autant donner une réponse claire à la question posée."

In [15]:
documents.sample().iloc[0].extractions

[{'sentiment': 'NEGATIVE',
  'extraction': 'Difficulté à trouver le bouton de déconnexion',
  'text': 'Où se trouve le bouton de déconnexion de vote site ?'},
 {'sentiment': 'SUGGESTION',
  'extraction': 'Bouton de déconnexion près du profil',
  'text': 'Il devrait se trouver prés de mon profil et me permettre de me déconnecter "proprement"'}]

In [16]:
documents.sample().iloc[0].splitted_analysis_v2

[{'text': 'Demande de passeport et de carte d’identité',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Demande de passeport et de carte d’identité'}]},
 {'text': 'J’avais des photos récentes on m’a demandé de les refaire sans aucun motif,',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Demande de refaire des photos sans motif'}]},
 {'text': 'sauf celui de payer à nouveau 6€.',
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Coût supplémentaire de 6€ pour les nouvelles photos'}]},
 {'text': "Ce n'est vraiment pas cool,",
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Insatisfaction concernant la demande de nouvelles photos'}]},
 {'text': "alors qu'elles avaient été acceptées au départ.",
  'extractions': [{'sentiment': 'NEGATIVE',
    'extraction': 'Photos initialement acceptées'}]},
 {'text': 'Pourquoi me refaire faire des photos ?',
  'extractions': [{'sentiment': 'SUGGESTION',
    'extraction': 'Question sur la nécessit